# Biosemi Active II recorded file streaming

In [32]:
# import mne
# import numpy as np
# from mne.channels import make_standard_montage
# import matplotlib.pyplot as plt
# from mne.datasets import eegbci

# raw = mne.io.read_raw_bdf("C:\\Users\\pipo_\\OneDrive\\Desktop\\test_data\\Testdata3.bdf")

# eegbci.standardize(raw)  # set channel names
# montage = make_standard_montage("biosemi64")

# raw.set_montage(montage, on_missing='ignore')

# raw_eeg = raw.copy().resample(512)

# raw_eeg.save("biosemi_SSVEP.fif", overwrite=True)

In [33]:
import mne
import numpy as np
from mne.channels import make_standard_montage
import matplotlib.pyplot as plt
from mne.datasets import eegbci


raw_eeg = mne.io.read_raw_fif("biosemi_SSVEP.fif")

Opening raw data file biosemi_SSVEP.fif...
    Range : 0 ... 100863 =      0.000 ...   196.998 secs
Ready.


C:\Users\pipo_\AppData\Local\Temp\ipykernel_24228\4293364053.py:8: RuntimeWarning: This filename (biosemi_SSVEP.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw_eeg = mne.io.read_raw_fif("biosemi_SSVEP.fif")


In [34]:
raw_eeg

<Raw | biosemi_SSVEP.fif, 73 x 100864 (197.0 s), ~87 kB, data not loaded>

In [35]:
events_raw = mne.find_events(raw_eeg, shortest_event = 0)

event_dict =  {'12Hz': 8,
'24Hz': 4,
'6Hz': 10,
'30Hz': 2,
'trial_begin': 1,
'trial_end': 20,
'end_exper': 40
}

Trigger channel STATUS has a non-zero initial value of {initial_value} (consider using initial_event=True to detect this event)
Removing orphaned offset at the beginning of the file.
61 events found on stim channel STATUS
Event IDs: [ 1  2  4  8 10 20 40]


In [36]:
events_raw

array([[ 9223,     0,     1],
       [ 9735,     0,     8],
       [12295,     0,    20],
       [13320,     0,     1],
       [13833,     0,     2],
       [16393,     0,    20],
       [17417,     0,     1],
       [17930,     0,     4],
       [20491,     0,    20],
       [21515,     0,     1],
       [22028,     0,    10],
       [24588,     0,    20],
       [25612,     0,     1],
       [26124,     0,    10],
       [28685,     0,    20],
       [29709,     0,     1],
       [30222,     0,     2],
       [32783,     0,    20],
       [33808,     0,     1],
       [34320,     0,     2],
       [36880,     0,    20],
       [37904,     0,     1],
       [38417,     0,    10],
       [40977,     0,    20],
       [42001,     0,     1],
       [42513,     0,    10],
       [45073,     0,    20],
       [46098,     0,     1],
       [46611,     0,     8],
       [49171,     0,    20],
       [50196,     0,     1],
       [50708,     0,    10],
       [53268,     0,    20],
       [54

In [37]:
from mne_realtime import LSLClient, MockLSLStream
from pylsl import StreamInfo, StreamOutlet
import time
import mne

# Assuming raw_eeg is already created and events are detected
# Find events in raw_eeg (already done in your case)
events_raw = mne.find_events(raw_eeg, shortest_event=0)

# Create the LSL stream for the trigger events (Markers)
trigger_info = StreamInfo(name='TriggerStream', type='Markers', channel_count=1, nominal_srate=raw_eeg.info['sfreq'], channel_format='int32')
trigger_outlet = StreamOutlet(trigger_info)

# Host name and wait time
host = 'My_mne_stream'
wait_max = 5

previous_event_time = 0

def send_triggers(events_raw, sfreq):
    global previous_event_time
    """ Send trigger events through LSL based on event timestamps. """
    for event in events_raw:
        sample_idx = event[0]  # The sample index of the event
        trigger_type = event[2]  # The event ID (trigger type)

        # Convert the sample index to time in seconds
        event_time = (sample_idx / sfreq)
        
        # Simulate waiting until the event time
        print("waiting to send event for: ",event_time-previous_event_time)
        time.sleep(event_time-previous_event_time)
        
        # Send the trigger over LSL
        trigger_outlet.push_sample([trigger_type])
        print(f"Sent trigger: {trigger_type} at time {event_time+previous_event_time:.3f} seconds")

        previous_event_time = event_time

if __name__ == '__main__':
    # Start the EEG stream using MockLSLStream
    with MockLSLStream(host, raw_eeg, 'eeg'):
        # Start the client to receive the EEG stream
        with LSLClient(info=raw_eeg.info, host=host, wait_max=wait_max) as client:
            client_info = client.get_measurement_info()
            
            # Send triggers in real-time based on the events found in the raw data
            send_triggers(events_raw, raw_eeg.info['sfreq'])

            # Infinite loop to simulate real-time streaming
            while True:
                pass  # Keeps the stream alive


Trigger channel STATUS has a non-zero initial value of {initial_value} (consider using initial_event=True to detect this event)
Removing orphaned offset at the beginning of the file.


61 events found on stim channel STATUS
Event IDs: [ 1  2  4  8 10 20 40]
Reading 0 ... 100863  =      0.000 ...   196.998 secs...
Client: Waiting for server to start
Looking for LSL stream My_mne_stream...
Found stream 'MNE' via My_mne_stream...
Client: Connected
waiting to send event for:  18.013671875
Sent trigger: 1 at time 18.014 seconds
waiting to send event for:  1.0
Sent trigger: 8 at time 37.027 seconds
waiting to send event for:  5.0
Sent trigger: 20 at time 43.027 seconds
waiting to send event for:  2.001953125
Sent trigger: 1 at time 50.029 seconds
waiting to send event for:  1.001953125
Sent trigger: 2 at time 53.033 seconds
waiting to send event for:  5.0
Sent trigger: 20 at time 59.035 seconds
waiting to send event for:  2.0
Sent trigger: 1 at time 66.035 seconds
waiting to send event for:  1.001953125
Sent trigger: 4 at time 69.037 seconds
waiting to send event for:  5.001953125
Sent trigger: 20 at time 75.041 seconds
waiting to send event for:  2.0
Sent trigger: 1 at ti

KeyboardInterrupt: 